In [1]:
import psycopg 
import pandas as pd

In [2]:
# делаем import необходимых библиотек
import os

import mlflow

# устанавливаем локальное хранилище для наших экспериментов
# хранилище должно быть такое же, как и при запуске сервиса
mlflow.set_tracking_uri('file:./mlflow_experiments_store')

# получаем id эксеримента, который создаётся по умолчанию
# эксперимент по умолчанию называется Default
experiment_id = mlflow.get_experiment_by_name("Default").experiment_id

# залогируем тестовую метрику и артефакт
with mlflow.start_run(run_name='Default', experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.log_metric("test_metric", 0)
    mlflow.log_artifact("test_artifact.txt", "test_artifact")

print(f"Run id запуска: {run_id}")

Run id запуска: 4b9a8cbd82d642d4ba2aaf58ff6d5a75


In [3]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": "rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net", 
    "port": "6432",
    "dbname": "playground_mle_20250130_582c662a4e",
    "user": "mle_20250130_582c662a4e",
    "password": "31992be741c54a3f85a88b4082cdcad7",
}
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "users_churn"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns)

In [4]:
import json
# 1. Название колонок вашего датафрейма запишите в текстовый файл
with open("columns.txt", "w", encoding="utf-8") as fio:
    fio.write(",".join(df.columns))
    
metrics = {
    "num_rows": len(df),
    "num_columns": len(df.columns),
}

with open("metrics.json", "w", encoding="utf-8") as f:
    json.dump(metrics, f, indent=4)
    
df.to_csv("dataset.csv", index=False)

In [5]:
counts_columns = [
    "type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"
]

stats = {}

for col in counts_columns:
    # Посчитать количество уникальных значений в колонке
    column_stat = df[col].value_counts().to_dict()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}

    # Обновить словарь stats
    stats.update(column_stat)

# Общая длина данных
stats["data_length"] = df.shape[0]

# Статистики по monthly_charges
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max()
stats["monthly_charges_mean"] = df["monthly_charges"].mean()
stats["monthly_charges_median"] = df["monthly_charges"].median()

# Статистики по total_charges
stats["total_charges_min"] = df["total_charges"].min()
stats["total_charges_max"] = df["total_charges"].max()
stats["total_charges_mean"] = df["total_charges"].mean()
stats["total_charges_median"] = df["total_charges"].median()

# Количество уникальных пользователей
stats["unique_customers_number"] = df["customer_id"].nunique()

# Количество пропусков в end_date
stats["end_date_nan"] = df["end_date"].isna().sum()

In [6]:
import mlflow
import os
import json

# задаём название эксперимента и имя запуска для логирования в MLflow
EXPERIMENT_NAME = "churn_fio"
RUN_NAME = "data_check"

# создаём новый эксперимент в MLflow с указанным названием 
# если эксперимент с таким именем уже существует, 
# MLflow возвращает идентификатор существующего эксперимента
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

# Проверка наличия файлов перед логированием и их создание, если их нет
columns_file_path = "columns.txt"
dataset_file_path = 'users_churn.csv'

# Проверка на существование файла и его создание
if not os.path.exists(columns_file_path):
    with open(columns_file_path, "w", encoding="utf-8") as f:
        f.write(",".join(df.columns))

if not os.path.exists(dataset_file_path):
    df.to_csv(dataset_file_path, index=False)

# Убедимся, что файлы существуют
assert os.path.exists(columns_file_path), f"{columns_file_path} does not exist."
assert os.path.exists(dataset_file_path), f"{dataset_file_path} does not exist."

# Начинаем логирование, передавая experiment_id
with mlflow.start_run(experiment_id=experiment_id, run_name=RUN_NAME) as run:
    run_id = run.info.run_id  # Получаем уникальный ID запуска
    
    # Логируем метрики
    mlflow.log_metrics(stats)

    # Логируем файлы как артефакты эксперимента, передавая абсолютный путь
    mlflow.log_artifact('columns.txt', artifact_path="dataframe")
    mlflow.log_artifact('users_churn.csv', artifact_path="dataframe")

# Получаем информацию о запуске
run = mlflow.get_run(run_id)

# Проверяем статус завершённого эксперимента
assert run.info.status == "FINISHED"

# Удаляем файлы после логирования
os.remove(columns_file_path)
os.remove(dataset_file_path)

In [7]:
EXPERIMENT_NAME = "churn_fio"

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

if experiment:
    experiment_id = experiment.experiment_id
    experiment_name = experiment.name
    run_count = len(mlflow.search_runs(experiment_ids=[experiment_id]))
    
    print(f"experiment_id = {experiment_id}")
    print(f"experiment_name = '{experiment_name}'")
    print(f"run_count = {run_count}")

experiment_id = 214559774650967674
experiment_name = 'churn_fio'
run_count = 1
